In [7]:
import pandas as pd
import numpy as np
import GPy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
data = pd.read_csv('sensor_data.csv')

# Impute the missing values using mean imputation
data['temperature'].fillna(data['temperature'].mean(), inplace=True)
data['humidity'].fillna(data['humidity'].mean(), inplace=True)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Scale and normalize the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data[['temperature', 'humidity']])
train_labels = np.array(train_data[['temperature', 'humidity']])

test_features = scaler.transform(test_data[['temperature', 'humidity']])
test_labels = np.array(test_data[['temperature', 'humidity']])

In [8]:
# Define the GPR model
kernel = GPy.kern.RBF(input_dim=2, variance=1., lengthscale=1.)
gpr_model = GPy.models.GPRegression(train_features, train_labels, kernel)

In [9]:
# Train the GPR model
gpr_model.optimize()

In [11]:
# Evaluate the GPR model
rmse = np.sqrt(np.mean((test_labels - gpr_model.predict(test_features)[0])**2))
print("RMSE:", rmse)

RMSE: 1.2810192648018627e-06


In [13]:
# Impute the missing values using the GPR model
missing_data = data.loc[data['temperature'].isnull() | data['humidity'].isnull()]
missing_data['temperature'].fillna(missing_data['temperature'].mean(), inplace=True)
missing_data['humidity'].fillna(missing_data['humidity'].mean(), inplace=True)

missing_features = scaler.transform(missing_data[['temperature', 'humidity']])
imputed_labels = gpr_model.predict(missing_features)[0]
missing_data[['temperature', 'humidity']] = imputed_labels

# Merge the imputed data back into the original data frame
data.update(missing_data)

# Show the first few rows of the imputed data
print(data.head())

 /var/folders/8d/ftv5hsns5r3gsjsltnwk_ysr0000gn/T/ipykernel_41674/1956145848.py:3: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /var/folders/8d/ftv5hsns5r3gsjsltnwk_ysr0000gn/T/ipykernel_41674/1956145848.py:4: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.